<a href="https://colab.research.google.com/github/dadisan/32602200053_DavidDimasSantana/blob/main/Proyek_Deteksi_Kendaraan_dengan_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Langkah 1: Instal library yang diperlukan
# Jalankan sel ini terlebih dahulu
!pip install ultralytics opencv-python

# Langkah 2: Impor semua library di awal
import cv2
from ultralytics import YOLO
from google.colab import drive
import os

# Langkah 3: Mount Google Drive Anda
drive.mount('/content/drive')

# Langkah 4: Siapkan path dan model
# Pastikan path ini benar sesuai dengan lokasi video di Google Drive Anda
video_input_path = '/content/drive/MyDrive/DatasetCCTV/12.mp4'
# Tentukan di mana Anda ingin menyimpan video hasil deteksi
video_output_path = '/content/drive/MyDrive/DatasetCCTV/hasil_deteksi_dengan_hitungan.mp4'

# Muat model YOLOv8
model = YOLO('yolov8n.pt')

# Daftar kelas kendaraan yang ingin dideteksi
kelas_kendaraan = ['car', 'motorcycle', 'bus', 'truck', 'bicycle']

# Langkah 5: Buka video sumber dan siapkan video output
cap = cv2.VideoCapture(video_input_path)
if not cap.isOpened():
    print(f"Error: Tidak dapat membuka video sumber di {video_input_path}")
else:
    # Dapatkan properti video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Tentukan codec dan buat objek VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_output_path, fourcc, fps, (frame_width, frame_height))

    # --- LOGIKA PENGHITUNGAN BARU ---
    # Tentukan posisi garis hitung (misalnya, di tengah vertikal)
    garis_hitung_y = int(frame_height * 0.5)

    # Dictionary untuk menyimpan hitungan setiap kelas kendaraan
    hitung_kendaraan = {nama: 0 for nama in kelas_kendaraan}

    # Set untuk menyimpan ID kendaraan yang sudah dihitung (agar tidak dihitung ganda)
    kendaraan_terhitung = set()
    # --------------------------------

    print("Memproses video dan menghitung kendaraan...")

    while cap.isOpened():
        sukses, frame = cap.read()
        if not sukses:
            break

        # Gambar garis hitung pada frame
        cv2.line(frame, (0, garis_hitung_y), (frame_width, garis_hitung_y), (0, 0, 255), 2)

        # Lakukan pelacakan objek (tracking)
        hasil = model.track(frame, persist=True)

        # Periksa apakah ada objek yang terdeteksi dan memiliki ID pelacakan
        if hasil[0].boxes.id is not None:
            kotak_pembatas = hasil[0].boxes.xyxy.cpu().numpy().astype(int)
            id_kelas = hasil[0].boxes.cls.cpu().numpy().astype(int)
            id_pelacakan = hasil[0].boxes.id.cpu().numpy().astype(int)

            for box, cls_id, track_id in zip(kotak_pembatas, id_kelas, id_pelacakan):
                nama_kelas = model.names[cls_id]

                if nama_kelas in kelas_kendaraan:
                    x1, y1, x2, y2 = box
                    # Gambar kotak pembatas
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    # Tulis label dengan ID pelacakan
                    label = f'ID: {track_id} {nama_kelas}'
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # --- LOGIKA PENGHITUNGAN BARU ---
                    # Hitung titik tengah objek
                    center_y = int((y1 + y2) / 2)

                    # Periksa apakah titik tengah objek melewati garis dan ID belum dihitung
                    if center_y > garis_hitung_y and track_id not in kendaraan_terhitung:
                        # Tambah hitungan untuk kelas kendaraan yang sesuai
                        hitung_kendaraan[nama_kelas] += 1
                        # Tambahkan ID ke set agar tidak dihitung lagi
                        kendaraan_terhitung.add(track_id)
                    # --------------------------------

        # --- TAMPILAN HITUNGAN BARU ---
        # Buat background semi-transparan untuk teks hitungan agar mudah dibaca
        overlay = frame.copy()
        cv2.rectangle(overlay, (5, 5), (350, 150), (0, 0, 0), -1)
        alpha = 0.6  # Tingkat transparansi
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        # Tampilkan hitungan setiap kendaraan
        y_offset = 30
        for nama, jumlah in hitung_kendaraan.items():
            teks_hitungan = f'{nama.capitalize()}: {jumlah}'
            cv2.putText(frame, teks_hitungan, (15, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            y_offset += 30
        # -----------------------------

        # Tulis frame yang sudah diproses ke file video output
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Video selesai diproses dan disimpan di: {video_output_path}")
    print("\n--- HASIL AKHIR PENGHITUNGAN ---")
    for nama, jumlah in hitung_kendaraan.items():
        print(f"{nama.capitalize()}: {jumlah}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Memproses video dan menghitung kendaraan...

0: 384x640 2 persons, 7 cars, 1 motorcycle, 2 trucks, 167.8ms
Speed: 3.7ms preprocess, 167.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 7 cars, 1 motorcycle, 2 trucks, 140.1ms
Speed: 4.6ms preprocess, 140.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6 cars, 1 motorcycle, 2 trucks, 141.7ms
Speed: 4.7ms preprocess, 141.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6 cars, 1 motorcycle, 2 trucks, 137.1ms
Speed: 4.4ms preprocess, 137.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 1 motorcycle, 3 trucks, 142.1ms
Speed: 4.6ms preprocess, 142.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6